In [1]:
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import os
from torch import optim

In [3]:
class SiameseNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SiameseNet, self).__init__()
        self.hidden_size = hidden_size
        
        self.Linear1 = nn.Linear(input_size, hidden_size)
        self.Linear2 = nn.Linear(hidden_size, hidden_size)
        self.Output = nn.Linear(hidden_size, 1)
        
    def forward_single(self, input):
        out = F.relu(self.Linear1(input))
        out = F.relu(self.Linear(out))
        out = self.Output(out)
        return out
    
    def forward(self, input1, input2, concat=False):
        out1 = self.forward_single(input1)
        out2 = self.forward_single(input2)
        if concat:
            return torch.cat([out1, out2], axis=1)
        return (out1, out2)
        

In [2]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [ ]:
class 